# **Simulazioni MD con il modulo CMS: CrystalPotential**

---

## Costruire un oggetto `CrystalPotential`

In `CMS`, il potenziale è rappresentato da un oggetto [CrystalPotential](../../CMS/MolecularDynamics/CrystalPotential.py), che per essere istanziato necessita di un oggetto `CrystalStructure`. Altri parametri che accetta il costruttore sono:

<div align="center">

| Attributo | Tipo | Default |
| :--- | :--- | :--- |
| **`sigma`** | float | 2.644 ($\AA$)|
| **`epsilon`** | float | 0.345 ($eV$) |
</div>

dove i valori di default sono adeguati per descrivere il potenziale di _Lennard-Jones_ in un cristallo di argento.

Come file .txt con le coordinate degli atomi si utilizza: [fcc100a256.txt](../../../data/pos-fcc100-random/fcc100a256.txt), dove 256 indica il numero `N` di atomi.

In [1]:
from CMS.MolecularDynamics.CrystalStructure import CrystalStructure
from CMS.MolecularDynamics.CrystalPotential import CrystalPotential

cristallo = CrystalStructure.from_file('../../../data/pos-fcc100-random/fcc100a256.txt')

potenziale = CrystalPotential(cristallo)

---

## Il calcolo dell'energia potenziale

### Il potenziale di Lennard-Jones

L'energia potenziale del sistema è calcolata come la somma delle singole energie di legame interatomiche:
$$
V = \frac12 \sum_{i\neq j \atop i,j=1}^{N} \phi(r_{ij})
$$
dove $\phi$ è il potenziale di Lennard-Jones:
$$
\phi_{ij}\equiv\phi(r_{ij}) = 4\varepsilon\left[
    \left( \frac{\sigma}{r_{ij}} \right)^{12} -
    \left( \frac{\sigma}{r_{ij}} \right)^{6}
    \right]
$$

<img src="../images/2-CrystalPotential/Lennard-Jones-potential.png" 
     style="display: block; margin-left: auto; margin-right: auto; margin-top: 5px; width: 400px;">

questa operazione è compiuta dal metodo `CrystalPotential.compute_potential()`

### Calcolo effettivo dell'energia potenziale

In [2]:
# calcolo dell'energia potenziale totale del cristallo
energia_potenziale = potenziale.compute_potential()
print(f"V = {energia_potenziale} eV")

V = -532.4800095250955 eV


Per poter calcolare il potenziale, è necessario riempire la matrice delle distanze interatomiche `CrystalStructure.distance_matrix`; se non dovesse essere già istanziata, `.compute_potential()` chiamerà `.find_neighbours()` del suo attributo `CrystalStructure` per costruirla da zero.

Poiché il potenziale di Lennard-Jones è un potenziale a _corto raggio_ $\int_{0}^{\infty} 4\pi r^2 \phi(r) dr < \infty$ è possibile approssimarlo considerando solamente le interazioni tra atomi sufficientemente vicini. Qui entra in gioco la determinazione dei vicini entro il raggio di _cutoff_ $R_C$.

> 💡 Se nell'oggetto `CrystalStructure` è stato impostato un raggio di _cutoff_ $R_C$, il potenziale di $\phi_{ij}$ è calcolato **solamente** tra le coppie di atomi considerati vicini tra loro, riducendo da $\mathcal{O}(N\times N)$ a $\mathcal{O}(N\times N_{neigh})$ il costo computazionale.

In [3]:
# aggiunta del raggio di cutoff: ⚠️ bisogna agire sull'attributo CrystalStructure di CrystalPotential
potenziale.crystal.set_R_C(4.5)
# .find_neighbours() va rieseguito, per sovrascrivere la matrice delle distanze, NxN -> NxN_neigh
potenziale.crystal.find_neighbours()
# calcolo dell'energia potenziale totale del cristallo con cutoff
energia_potenziale = potenziale.compute_potential()
print(f"V = {energia_potenziale} eV")

V = -453.4501806493874 eV


---

## Il calcolo delle forze

### Derivazione della forza $\vec{F}$

La forza sul $k$-esimo atomo è la derivata parziale del potenziale rispetto al vettore posizione dell'atomo stesso.
$$
\vec{F}_k(\{r_{ij}\}_{ij}) = -\vec{\nabla}_k\ V(\{r_{ij}\}_{ij}) = 
-\left(
\frac{\partial V}{\partial x_k}, 
\frac{\partial V}{\partial y_k}, 
\frac{\partial V}{\partial z_k}
\right)
$$
e.g. la componente $x$ della forza sul $k$-esimo atomo è:
$$
{F_k}_x = -\frac12\frac{\partial}{\partial x_k}\sum_{i\neq j} \phi(r_{ij}) =
-\frac12 \sum_{i\neq j} \frac{\partial\phi(r_{ij})}{\partial x_k}
$$
usando la *chain rule*:
$$
{F_k}_x = -\frac12\sum_{i\neq j}\frac{\partial\phi(r_{ij})}{\partial x_k} = 
-\frac12\sum_{i\neq j}\frac{\partial\, r_{ij}}{\partial x_k}
\frac{\partial\phi(r_{ij})}{\partial\, r_{ij}}
$$
dove:
$$
\begin{align*}
\frac{\partial\, r_{ij}}{\partial x_k} &= \frac{\partial}{\partial x_k}
\left[ (x_i-x_j)^2+(y_i-y_j)^2+(z_i-z_j)^2 \right]^{\frac12}=\\ 
&= 2(x_i-x_j)\frac{1}{2}\left[ (x_i-x_j)^2+(y_i-y_j)^2+(z_i-z_j)^2 \right]^{-\frac12}\delta_{ik}=\\
&= \frac{(x_i-x_j)}{r_{ij}}\delta_{ik}
\end{align*}
$$
e: 
$$
\begin{align*}
\frac{\partial\phi(r_{ij})}{\partial r_{ij}} &= \frac{\partial}{\partial r_{ij}}
4\varepsilon\left[
\left( \frac{\sigma}{r_{ij}} \right)^{12} -
\left( \frac{\sigma}{r_{ij}} \right)^{6}
\right] =\\
&=
4\varepsilon\left[
-\frac{12\sigma}{r_{ij}^{2}}\left( \frac{\sigma}{r_{ij}} \right)^{11} +
\frac{6\sigma}{r_{ij}^{2}}\left( \frac{\sigma}{r_{ij}} \right)^{5}
\right] =\\
&= 
4\varepsilon\left[
-12\left( \frac{\sigma^{12}}{r_{ij}^{13}} \right) +
6\left( \frac{\sigma^{6}}{r_{ij}^{7}} \right)
\right]
\end{align*}
$$

Infine:
$$
-\frac12 \sum_{i\neq j} \frac{\partial\phi(r_{ij})}{\partial x_k} = 
-\sum_{j\neq k}\ '\ 2\varepsilon \frac{(x_k-x_j)}{r_{kj}} \left[
-12\left( \frac{\sigma^{12}}{r_{kj}^{13}} \right) +
6\left( \frac{\sigma^{6}}{r_{kj}^{7}} \right)
\right]
$$
o, scritto in maniera più compatta:
$$
F_{kx} = 24\sigma^6\varepsilon\sum_{i\neq k} \frac{1}{r_{ik}^8}
\left[
\frac{2\sigma^6}{r_{ik}^6}-1
\right]
\left(
x_i-x_k
\right)
$$

### Calcolo effettivo di $\{\vec{F}_k\}_{k=0,\dots,N-1}$

In `CMS` la forza è una matrice $N\times3$, in cui la $k$-esima riga contiene il vettore forza dell'atomo $k$-esimo

In [4]:
matrice_forza = potenziale.compute_forces()
print(matrice_forza)

[[ 1.13839380e-01  1.13839380e-01 -1.13110266e-01]
 [ 3.64557175e-04  3.64557175e-04  3.64557175e-04]
 [-5.63728542e-02 -2.42167397e-15 -5.63728542e-02]
 [ 1.13839380e-01  5.71764858e-15 -1.13110266e-01]
 [-5.63728542e-02 -3.76088050e-15 -5.63728542e-02]
 [ 1.13839380e-01 -1.80411242e-16 -1.13110266e-01]
 [-5.63728542e-02 -1.13839380e-01 -5.63728542e-02]
 [ 1.13839380e-01  5.63728542e-02 -5.63728542e-02]
 [-2.42167397e-15 -5.63728542e-02 -5.63728542e-02]
 [ 5.71764858e-15  1.13839380e-01 -1.13110266e-01]
 [-2.42167397e-15 -2.40779618e-15 -1.13110266e-01]
 [ 5.22498711e-15  5.21804822e-15 -1.13110266e-01]
 [-1.94982919e-15 -4.95437025e-15 -1.13110266e-01]
 [ 4.20496971e-15 -1.80411242e-16 -1.13110266e-01]
 [-1.18655086e-15 -1.13839380e-01 -1.13110266e-01]
 [ 3.02535774e-15  5.63728542e-02 -5.63728542e-02]
 [-3.76088050e-15 -5.63728542e-02 -5.63728542e-02]
 [-1.80411242e-16  1.13839380e-01 -1.13110266e-01]
 [-4.95437025e-15 -1.94982919e-15 -1.13110266e-01]
 [-1.80411242e-16  4.19803081e-

Ogni entrata di `matrice_forza` è un `np.ndarray` delle tre componenti (x, y, z) della forza agente sull'atomo corrispondente. Ad esempio, `matrice_forza[0]` contiene la somma delle forze esercitate sul primo atomo dai suoi vicini `[Fx, Fy, Fz]`.

---

## La giunzione polinomiale `PolynomialJunction`

Quando viene introdotto il raggio di cutoff $R_C$, il potenziale di Lennard-Jones viene tagliato in modo brusco per tutte le distanze $r_{ij}>R_C$.

$$
\phi(r_{ij}) = 
\begin{cases}
    4\varepsilon\left[
    \left( \frac{\sigma}{r_{ij}} \right)^{12} -
    \left( \frac{\sigma}{r_{ij}} \right)^{6}
    \right], 
    &\text{ se } 0 < r_{ij} < R_C
    \\
    0, 
    &\text{ se } r_{ij} > R_C
\end{cases}
$$

<img src="../images/2-CrystalPotential/Lennard-Jones-cutoff.png" 
     style="display: block; margin-left: auto; margin-right: auto; margin-top: 5px; width: 400px;">

Ciò risulta determinante, in negativo, quando si eseguono simulazioni di dinamica: le equazioni del moto di Newton per un'hamiltioniana indipendente dal tempo portano alla conservazione dell'energia solo se il potenziale si comporta opportunamente.
Per evitare questa discontinuità così marcata, è possibile introdurre una **giunzione polinomiale**, tramite un oggetto `PolynomialJunction`. Esso introduce nella figura del potenziale un polinomio di settimo grado, garantendo che l'interazione raggiunga lo 0 più dolcemente di un semplice taglio. Il polinomio viene applicato a tutte le ditanze comprese tra un'arbitraria distanza $R_P$ (**raggio di giunzione**) e il raggio di _cutoff_:

$$
V(r_{ij}) = 
\begin{cases}
    \phi(r_{ij}) = 4\varepsilon\left[
    \left( \frac{\sigma}{r_{ij}} \right)^{12} -
    \left( \frac{\sigma}{r_{ij}} \right)^{6}
    \right], 
    &\text{ se } 0 < r_{ij} < R_P
    \\
    P_7(r_{ij}) = Ar_{ij}^{7} + Br_{ij}^{6} + Cr_{ij}^{5} + Dr_{ij}^{4} + Er_{ij}^{3} + Fr_{ij}^{2} + Gr_{ij} + H,
    &
    \text{ se } R_P < r_{ij} < R_C
    \\
    0, 
    &\text{ se } r_{ij} > R_C
\end{cases}
$$

Per farlo, è necessario impostare nell'oggetto `CrystalStructure` un **raggio di giunzione** $R_P$, quindi, al momento della creazione di un `CrystalPotential`, comunicargli che vogliamo inserire la giunzione polinomiale. L'oggetto `PolynomialJunction`, alla creazione, calcola i coefficienti $\{A,...,H\}$ adatti a congiungere le regioni $R_P$ e $R_C$ di un potenziale LJ descritto da $\sigma$ e $\varepsilon$; necessita pertanto dei seguenti parametri:

<div align="center">

| Attributo | Tipo | Default |
| :--- | :--- | :--- |
| **`R_C`** | float | /|
| **`R_P`** | float | /|
| **`sigma`** | float | 2.644 ($\AA$)|
| **`epsilon`** | float | 0.345 ($eV$) |
</div>

In [5]:
# oggetto CrystalStructure con raggio di cutoff e raggio di giunzione
cristallo = CrystalStructure.from_file('../../../data/pos-fcc100-random/fcc100a256.txt')
cristallo.set_R_C(4.5)
cristallo.set_R_P(4.2)

# oggetto PolynomialJunction
from CMS.MolecularDynamics.PolynomialJunction import PolynomialJunction
giunzione = PolynomialJunction(
    R_C = cristallo.R_C,
    R_P = cristallo.R_P
)

# oggetto CrystalPotential con giunzione polinomiale
potenziale_con_giunzione = CrystalPotential(crystal=cristallo, poly7=giunzione)

> ⚠️ L'attributo `poly7` di `CrystalPotential` è `None` di default, va quindi fornito come parametro _keyword_ se si vuole implementare, altrimenti verrà ignorato.

Per un potenziale con giunzione valgono i discorsi fatti in precedenza, con la differenza che, nella regione $R_P<r<R_C$ la forza prenderà in considerazione la derivata del polinomio (un polinomio di grado 6), invece del potenziale LJ.

$$
F_{ki_{x}} = 
\begin{cases}
    24\sigma^6\varepsilon \frac{1}{r_{ik}^8}
    \left[
    \frac{2\sigma^6}{r_{ik}^6}-1
    \right]
    \left(
    x_i-x_k
    \right),
    &\text{ se } 0 < r_{ki} < R_P
    \\
    7Ar_{ki}^{6} + 6Br_{ki}^{5} + 5Cr_{ki}^{4} + 4Dr_{ki}^{3} + 3Er_{ki}^{2} + 2Fr_{ki}^{} + G
    \frac{\left(x_i-x_k\right)}{r_{ki}},
    &\text{ se } R_P < r_{ki} < R_C
    \\
    0,
    &\text{ se } r_{ki} > R_C
\end{cases}
$$

Quindi si possono chiamare i metodi `.compute_potential()` e `.compute_forces()` per il nuovo `potenziale_con_giunzione`:

In [6]:
# calcolo dell'energia potenziale totale del cristallo con giunzione polinomiale
energia_potenziale_con_giunzione = potenziale_con_giunzione.compute_potential()
print(f"V con giunzione polinomiale = {energia_potenziale_con_giunzione} eV")

# calcolo delle forze con giunzione polinomiale
matrice_forza_con_giunzione = potenziale_con_giunzione.compute_forces()

V con giunzione polinomiale = -453.4501806493874 eV


<img src="../images/2-CrystalPotential/polynomial-junction.png" 
     style="display: block; margin-left: auto; margin-right: auto; margin-top: 5px; width: 400px;">